# Prac W8
## MLPS and Convolutional Neural Networks (CNNs) using Tensorflow 2.x

A lot of this will be borrowed from the Tensorflow introduction found [here](https://www.tensorflow.org/tutorials/)

You've already covered multilayer perceptrons in last weeks prac. CNNs are possibly part of the reason you're interested in this course due to their strengths in image classification.

[This link](https://cs231n.github.io/convolutional-networks/) provides a good overview of CNNs and would be useful to read. 

Some useful imports 

In [1]:
import tensorflow as tf
%load_ext tensorboard
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv3D, MaxPooling2D
from tensorflow.keras import Model
import datetime
import numpy as np
!rm -rf ./logs/ 
print(tf.__version__) #Double check the colab has the instance of tensorflow we want

2.16.1


# Import the CIFAR10 Dataset

In [2]:
# Load the CIFAR10 dataset
cifar10 = tf.keras.datasets.cifar10

(cifar_x_train, cifar_y_train), (cifar_x_test, cifar_y_test) = cifar10.load_data()

cifar_x_train = cifar_x_train / 255
cifar_x_test = cifar_x_test / 255


### Explore the dataset shape
### Create tensor batches

In [3]:
print("Image count:")
print(len(cifar_x_train))
print("Pixels dimension 1:")
print(len(cifar_x_train[0]))
print("Pixels dimension 2:")
print(len(cifar_x_train[0][0]))
print("RGB Dimensions:")
print(len(cifar_x_train[0][0][0]))


#cifar_x_train = cifar_x_train.astype(np.float32)
#cifar_x_test = cifar_x_test.astype(np.float32)

cifar_train_ds = tf.data.Dataset.from_tensor_slices(
    (cifar_x_train, cifar_y_train)
).shuffle(10000).batch(32)

cifar_test_ds = tf.data.Dataset.from_tensor_slices(
    (cifar_x_test, cifar_y_test)
).batch(32)


Image count:
50000
Pixels dimension 1:
32
Pixels dimension 2:
32
RGB Dimensions:
3


In [4]:
class ModelTester:
  def __init__(
      self,
      model_name,
      model, 
      optimiser = None,
      loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  ):
    self._model = model
    self.model_name = model_name
    if optimiser is None:
        self._optimiser = tf.keras.optimizers.Adam() 
    self._loss_object = loss_object
    self._train_loss = tf.keras.metrics.Mean(name='train_loss')
    self._train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    self._test_loss = tf.keras.metrics.Mean(name='test_loss')
    self._test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
  
  @tf.function
  def train_step(self, images, labels):
    with tf.GradientTape() as tape:
      # training=True is only needed if there are layers with different
      # behavior during training versus inference (e.g. Dropout).
      predictions = self._model(images, training=True)
      loss = self._loss_object(labels, predictions)
    gradients = tape.gradient(loss, self._model.trainable_variables)
    self._optimiser.apply_gradients(zip(gradients, self._model.trainable_variables))

    self._train_loss(loss)
    self._train_accuracy(labels, predictions) 

  @tf.function
  def test_step(self, images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = self._model(images, training=False)
    t_loss = self._loss_object(labels, predictions)

    self._test_loss(t_loss)
    self._test_accuracy(labels, predictions)

  def train(self, train_ds, test_ds, epochs):
    for epoch in range(epochs):
      # Reset the metrics at the start of the next epoch
      self._train_loss.reset_state()
      self._train_accuracy.reset_state()
      self._test_loss.reset_state()
      self._test_accuracy.reset_state()

      for images, labels in train_ds:
        self.train_step(images, labels)

      for test_images, test_labels in test_ds:
        self.test_step(test_images, test_labels)

      template = 'Model: {}, Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
      print(template.format(
        self.model_name,
        epoch + 1,
        self._train_loss.result(),
        self._train_accuracy.result() * 100,
        self._test_loss.result(),
        self._test_accuracy.result() * 100)
      )

In [5]:
model =  tf.keras.models.Sequential([
            Conv2D(32,(3,3)), 
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #datetime storage
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) #TB callbacks

#model.fit(cifar_train_ds, epochs=5, validation_data=cifar_test_ds, callbacks=[tensorboard_callback])

In [6]:

model_single_convolutional_layer =  tf.keras.models.Sequential([
            Conv2D(32,3),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_convolutional_layer = ModelTester('Single Convolution layer', model_single_convolutional_layer)
tester_convolutional_layer.train(cifar_train_ds, cifar_test_ds, 5)


2024-04-24 17:16:26.214823: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:16:28.876813: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer, Epoch: 1, Loss: 1.7240190505981445, Accuracy: 41.02799987792969, Test Loss: 1.5022093057632446, Test Accuracy: 46.5099983215332


2024-04-24 17:17:27.004255: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:17:29.524253: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer, Epoch: 2, Loss: 1.4407844543457031, Accuracy: 49.571998596191406, Test Loss: 1.446818232536316, Test Accuracy: 48.98999786376953


2024-04-24 17:18:27.120727: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:18:29.650084: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 17:19:29.010187: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:19:31.633864: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer, Epoch: 4, Loss: 1.173329472541809, Accuracy: 58.49599838256836, Test Loss: 1.494692325592041, Test Accuracy: 49.86000061035156


2024-04-24 17:20:30.787413: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer, Epoch: 5, Loss: 1.0471221208572388, Accuracy: 62.884002685546875, Test Loss: 1.5397394895553589, Test Accuracy: 50.19999694824219


2024-04-24 17:20:33.385348: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:

model_single_convolutional_layer_no_softmax =  tf.keras.models.Sequential([
            Conv2D(32,3),
            Flatten(),
            Dense(512, activation='relu')
          ])

tester_convolutional_layer_no_softmax = ModelTester('Single Convolution layer no softmax', model_single_convolutional_layer_no_softmax)
tester_convolutional_layer_no_softmax.train(cifar_train_ds, cifar_test_ds, 5)


2024-04-24 14:36:48.130679: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 14:36:50.675118: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer no softmax, Epoch: 1, Loss: 11.073408126831055, Accuracy: 12.22599983215332, Test Loss: 11.05459976196289, Test Accuracy: 12.850000381469727


2024-04-24 14:37:49.452324: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 14:37:52.267073: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer no softmax, Epoch: 2, Loss: 11.056304931640625, Accuracy: 12.605998992919922, Test Loss: 11.05459976196289, Test Accuracy: 10.0


2024-04-24 14:38:49.940490: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 14:38:52.462239: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 14:39:51.459090: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 14:39:53.986989: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer no softmax, Epoch: 4, Loss: 11.056305885314941, Accuracy: 10.0, Test Loss: 11.05459976196289, Test Accuracy: 10.0


2024-04-24 14:40:54.909396: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer no softmax, Epoch: 5, Loss: 11.057280540466309, Accuracy: 10.0, Test Loss: 11.05459976196289, Test Accuracy: 10.0


2024-04-24 14:40:57.699545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:

model_single_convolutional_layer_4_4 =  tf.keras.models.Sequential([
            Conv2D(
                filters=16,
                kernel_size=(4,4),
                strides=2
            ),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_convolutional_layer_4_4 = ModelTester('Single Convolution layer 4x4 kernal', model_single_convolutional_layer_4_4)
tester_convolutional_layer_4_4.train(cifar_train_ds, cifar_test_ds, 5)


2024-04-24 17:21:31.034587: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:21:31.737612: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 1, Loss: 1.6199595928192139, Accuracy: 42.99399948120117, Test Loss: 1.4495594501495361, Test Accuracy: 48.66999816894531


2024-04-24 17:21:43.982471: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:21:44.517646: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 2, Loss: 1.3273426294326782, Accuracy: 53.7599983215332, Test Loss: 1.3544358015060425, Test Accuracy: 52.22999954223633


2024-04-24 17:21:57.069302: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:21:57.593281: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 17:22:10.110391: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:22:10.679361: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 4, Loss: 1.001168966293335, Accuracy: 64.93000030517578, Test Loss: 1.4818010330200195, Test Accuracy: 52.29999923706055


2024-04-24 17:22:23.353897: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 5, Loss: 0.833291232585907, Accuracy: 70.81999969482422, Test Loss: 1.5196995735168457, Test Accuracy: 53.3900032043457


2024-04-24 17:22:23.896562: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:

model_single_convolutional_layer_8_8 =  tf.keras.models.Sequential([
            Conv2D(
                filters=64,
                kernel_size=(8,8),
                strides=3
            ),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_convolutional_layer_8_8 = ModelTester('Single Convolution layer 4x4 kernal', model_single_convolutional_layer_8_8)
tester_convolutional_layer_8_8.train(cifar_train_ds, cifar_test_ds, 5)


2024-04-24 17:24:47.872204: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:24:48.666953: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 1, Loss: 1.6983232498168945, Accuracy: 39.80999755859375, Test Loss: 1.5204046964645386, Test Accuracy: 45.849998474121094


2024-04-24 17:25:03.628237: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:25:04.386835: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 2, Loss: 1.482574224472046, Accuracy: 47.43199920654297, Test Loss: 1.4567539691925049, Test Accuracy: 48.209999084472656


2024-04-24 17:25:19.516685: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:25:20.281192: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 17:25:35.474481: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:25:36.219230: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 4, Loss: 1.3375849723815918, Accuracy: 53.06800079345703, Test Loss: 1.4419726133346558, Test Accuracy: 48.849998474121094


2024-04-24 17:25:51.111189: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 5, Loss: 1.284733533859253, Accuracy: 54.73999786376953, Test Loss: 1.3889940977096558, Test Accuracy: 51.75


2024-04-24 17:25:51.857116: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:

model_single_convolutional_layer_8_8 =  tf.keras.models.Sequential([
            Conv2D(
                filters=64,
                kernel_size=(4,4),
                strides=2
            ),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_convolutional_layer_8_8 = ModelTester('Single Convolution layer 4x4 kernal', model_single_convolutional_layer_8_8)
tester_convolutional_layer_8_8.train(cifar_train_ds, cifar_test_ds, 5)


2024-04-24 17:26:33.491787: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:26:35.093064: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 1, Loss: 1.6660102605819702, Accuracy: 41.6359977722168, Test Loss: 1.4648007154464722, Test Accuracy: 47.53000259399414


2024-04-24 17:27:03.095029: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:27:04.452418: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 2, Loss: 1.4119011163711548, Accuracy: 50.540000915527344, Test Loss: 1.434283971786499, Test Accuracy: 48.71999740600586


2024-04-24 17:27:34.780783: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:27:36.180589: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 17:28:04.052350: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:28:05.435720: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 4, Loss: 1.1684685945510864, Accuracy: 58.76799774169922, Test Loss: 1.461327314376831, Test Accuracy: 51.13999938964844


2024-04-24 17:28:33.303102: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer 4x4 kernal, Epoch: 5, Loss: 1.056992769241333, Accuracy: 62.91400146484375, Test Loss: 1.555098056793213, Test Accuracy: 51.16999816894531


2024-04-24 17:28:34.700561: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:


model_two_convolutional_layers =  tf.keras.models.Sequential([
            Conv2D(
                filters=16,
                kernel_size=(4,4),
                strides=2
            ),
            Conv2D(
                filters=16,
                kernel_size=(4,4),
                strides=2
            ),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_two_convolutional_layers = ModelTester('Two Convolution layers', model_two_convolutional_layers)
tester_two_convolutional_layers.train(cifar_train_ds, cifar_test_ds, 5)


2024-04-24 17:28:46.946266: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:28:47.480441: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Two Convolution layers, Epoch: 1, Loss: 1.5961720943450928, Accuracy: 43.63600158691406, Test Loss: 1.4010000228881836, Test Accuracy: 50.290000915527344


2024-04-24 17:28:54.421931: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:28:54.854133: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Two Convolution layers, Epoch: 2, Loss: 1.3323668241500854, Accuracy: 53.65599822998047, Test Loss: 1.3152687549591064, Test Accuracy: 54.220001220703125


2024-04-24 17:29:01.923209: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:29:02.383469: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 17:29:09.349524: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:29:09.806262: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Two Convolution layers, Epoch: 4, Loss: 1.0842912197113037, Accuracy: 62.60600280761719, Test Loss: 1.3146229982376099, Test Accuracy: 56.08000183105469


2024-04-24 17:29:16.841589: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Two Convolution layers, Epoch: 5, Loss: 0.9659060835838318, Accuracy: 66.60400390625, Test Loss: 1.3455389738082886, Test Accuracy: 55.659996032714844


2024-04-24 17:29:17.291295: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:

model_single_convolutional_layer_multiple_dense_layers =  tf.keras.models.Sequential([
            Conv2D(
                filters=16,
                kernel_size=(4,4),
                strides=2
            ),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(512, activation='relu'),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_convolutional_layer_multiple_dense = ModelTester('Single Convolution layer multiple dense layers', model_single_convolutional_layer_multiple_dense_layers)
tester_convolutional_layer_multiple_dense.train(cifar_train_ds, cifar_test_ds, 5)

2024-04-24 17:30:36.412929: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:30:37.511369: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer multiple dense layers, Epoch: 1, Loss: 1.6189134120941162, Accuracy: 41.68600082397461, Test Loss: 1.4198156595230103, Test Accuracy: 49.56999969482422


2024-04-24 17:30:56.786955: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:30:57.746823: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer multiple dense layers, Epoch: 2, Loss: 1.3170162439346313, Accuracy: 53.04999923706055, Test Loss: 1.4043800830841064, Test Accuracy: 51.130001068115234


2024-04-24 17:31:17.528632: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:31:18.546621: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 17:31:38.500805: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:31:39.587721: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer multiple dense layers, Epoch: 4, Loss: 0.9829220175743103, Accuracy: 64.43000030517578, Test Loss: 1.399260401725769, Test Accuracy: 53.32999801635742


2024-04-24 17:32:00.855117: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer multiple dense layers, Epoch: 5, Loss: 0.8172318935394287, Accuracy: 70.47000122070312, Test Loss: 1.5065793991088867, Test Accuracy: 52.6400032043457


2024-04-24 17:32:01.727095: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
model_single_convolutional_layer_multiple_small_dense_layers =  tf.keras.models.Sequential([
            Conv2D(
                filters=16,
                kernel_size=(4,4),
                strides=2
            ),
            Flatten(),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_convolutional_layer_multiple_small_dense = ModelTester('Single Convolution layer many small dense layers', model_single_convolutional_layer_multiple_small_dense_layers)
tester_convolutional_layer_multiple_small_dense.train(cifar_train_ds, cifar_test_ds, 10)


2024-04-24 17:32:16.398748: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:32:16.810590: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer many small dense layers, Epoch: 1, Loss: 1.9244521856307983, Accuracy: 27.323999404907227, Test Loss: 1.7336710691452026, Test Accuracy: 36.2599983215332


2024-04-24 17:32:22.933028: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:32:23.220984: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer many small dense layers, Epoch: 2, Loss: 1.6873931884765625, Accuracy: 38.672000885009766, Test Loss: 1.648781657218933, Test Accuracy: 40.459999084472656


2024-04-24 17:32:29.445547: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:32:29.753891: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 17:33:07.965642: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 17:33:08.237298: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer many small dense layers, Epoch: 9, Loss: 1.3734114170074463, Accuracy: 51.275997161865234, Test Loss: 1.5083426237106323, Test Accuracy: 47.09000015258789


2024-04-24 17:33:14.257088: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: Single Convolution layer many small dense layers, Epoch: 10, Loss: 1.3580316305160522, Accuracy: 51.69600296020508, Test Loss: 1.563693642616272, Test Accuracy: 44.53000259399414


2024-04-24 17:33:14.546693: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [23]:
model_no_convolutional_layer =  tf.keras.models.Sequential([
            Flatten(),
            Dense(512, activation='relu'),
            Dense(512, activation='relu'),
            Dense(512, activation='relu'),
            Dense(512, activation='relu'),
            Dense(512, activation='relu'),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_no_convolutional_layer_model = ModelTester('No convolutional layer', model_no_convolutional_layer)
tester_no_convolutional_layer_model.train(cifar_train_ds, cifar_test_ds, 10)


2024-04-24 15:07:57.420545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:07:58.447673: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: No convolutional layer, Epoch: 1, Loss: 1.96039879322052, Accuracy: 26.36600112915039, Test Loss: 1.8567473888397217, Test Accuracy: 30.860000610351562


2024-04-24 15:08:23.021842: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:08:23.810870: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: No convolutional layer, Epoch: 2, Loss: 1.8256311416625977, Accuracy: 32.97599792480469, Test Loss: 1.7653164863586426, Test Accuracy: 35.91999816894531


2024-04-24 15:08:48.308693: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:08:49.307763: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 15:11:12.954933: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:11:13.978923: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: No convolutional layer, Epoch: 9, Loss: 1.589856505393982, Accuracy: 42.672000885009766, Test Loss: 1.5954020023345947, Test Accuracy: 42.54999923706055


2024-04-24 15:11:38.314791: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: No convolutional layer, Epoch: 10, Loss: 1.5736645460128784, Accuracy: 43.461997985839844, Test Loss: 1.5947819948196411, Test Accuracy: 43.13999938964844


2024-04-24 15:11:39.373355: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
model_single_convolutional_layer_many_many_small_dense_layers =  tf.keras.models.Sequential([
            Conv2D(32,3),
            Flatten(),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(10, activation='softmax')
          ])

tester_convolutional_layer_model_single_convolutional_layer_many_many_small_dense_layers = ModelTester('wow, many layers', model_single_convolutional_layer_multiple_small_dense_layers)
tester_convolutional_layer_model_single_convolutional_layer_many_many_small_dense_layers.train(cifar_train_ds, cifar_test_ds, 20)


2024-04-24 15:18:12.499756: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:18:13.394909: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: wow, many layers, Epoch: 1, Loss: 1.0252810716629028, Accuracy: 63.81599807739258, Test Loss: 1.7722394466400146, Test Accuracy: 44.96000289916992


2024-04-24 15:18:26.740072: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:18:27.477903: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: wow, many layers, Epoch: 2, Loss: 1.0155869722366333, Accuracy: 64.07600402832031, Test Loss: 1.7980457544326782, Test Accuracy: 44.90999984741211


2024-04-24 15:18:40.944914: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:18:41.743893: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-24 15:22:17.092631: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-24 15:22:17.752950: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: wow, many layers, Epoch: 19, Loss: 0.8911221623420715, Accuracy: 69.01399993896484, Test Loss: 1.9464199542999268, Test Accuracy: 45.09000015258789


2024-04-24 15:22:30.227993: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Model: wow, many layers, Epoch: 20, Loss: 0.880838930606842, Accuracy: 69.10200500488281, Test Loss: 1.9638326168060303, Test Accuracy: 43.88999938964844


2024-04-24 15:22:30.939129: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
print("DONE!")

DONE!
